<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/gte-large_LangChain_ChromaDB_Llama2-7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Step 1: Install All the Required Packages and Libraries**

In [1]:
%%capture
!pip install huggingface_hub
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install -q -U transformers peft accelerate optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install GPUtil

In [2]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.retrievers import ParentDocumentRetriever
from langchain.vectorstores import Chroma
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from textwrap import fill
from huggingface_hub import hf_hub_download
import GPUtil
import requests

#**Step 1A: Recursively split by character**

https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

Thu Jan 18 11:30:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
print('Initial GPU Usage:')
GPUtil.showUtilization()

Initial GPU Usage:
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [5]:
text = requests.get('https://raw.githubusercontent.com/winterForestStump/thesis/main/data/txt_examp_ver2.txt').text

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=32,
    length_function=len,
    is_separator_regex=False)

texts = text_splitter.create_documents([text])

print('GPU Usage after text splitting:')
GPUtil.showUtilization()

GPU Usage after text splitting:
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [7]:
len(texts)

1079

#**Step 1B: Embedding**

In [8]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

#Limitation
#This model exclusively caters to English texts, and any lengthy texts will be truncated to a maximum of 512 tokens.

print('GPU Usage after embedding:')
GPUtil.showUtilization()

GPU Usage after embedding:
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [9]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("What was the revenue of the company in 2023 and how it changed since 2021?", k=8)
print(results[0].page_content)

In fiscal year 2023, we achieved earnings growth through pricing discipline in our merchant business as well as improved on-site volumes, including higher demand for hydrogen, despite inflation, higher maintenance activities, and higher costs to support our long-term strategy. Due to the structure of our contracts, which generally contain fixed monthly charges and/or minimum purchase requirements, our on-site business generates stable cash flow and consistently contributes about half our total


In [10]:
print('GPU Usage after chroma DB:')
GPUtil.showUtilization()

GPU Usage after chroma DB:
| ID | GPU | MEM |
------------------
|  0 |  2% | 20% |


#**Step 2: Download the Model**

In [11]:
MODEL_NAME = "TheBloke/Llama-2-7b-Chat-GPTQ"
# Context Length: 4096

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto")

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
print('GPU Usage after LLM and tokenizer initialization:')
GPUtil.showUtilization()

GPU Usage after LLM and tokenizer initialization:
| ID | GPU | MEM |
------------------
|  0 |  0% | 36% |


Creating a plot template

In [13]:
template = """
<s>[INST] <<SYS>>
Act like an expert in financial analysis and accounting. Use the following information from company annual reports and answer the question at the end.
If you don't know the answer, don't make up an answer, but answer "I don't know".
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 8}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [14]:
print('GPU Usage after RetrievalQA:')
GPUtil.showUtilization()

GPU Usage after RetrievalQA:
| ID | GPU | MEM |
------------------
|  0 |  0% | 36% |


In [15]:
result = qa_chain(
    "What was the revenue (sales) of the company in 2023 and how it changed since 2021?"
)
print(fill(result["result"].strip(), width=90))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Based on the provided information, here are the answers to your questions: Revenue (sales)
of the company in 2023: $12.6 billion Change in revenue (sales) between 2021 and 2023: In
2021, the company's sales were $970.8 million, which decreased by 8% to $889.0 million in
2023.


In [16]:
print('GPU Usage after the retrieval 1st prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 1st prompt:
| ID | GPU | MEM |
------------------
|  0 | 70% | 44% |


other prompts:

In [17]:
result = qa_chain(
    "Summarize the financial results (revenue, costs and net incomes) of the company in 2023 and possible risks which can influence the results in the future in 10-12 sentences."
)
print(fill(result["result"].strip(), width=90))

Based on the provided financial statements, here are the key financial results of the
company in 2023: Revenue: The company generated $5,192.9 million in net sales in 2023,
representing a decrease of $1,031.5 million compared to 2022. Gross profit was $2,465.5
million, down by $233.4 million from the previous year. Costs: Total operating expenses
were $1,145.7 million in 2023, an increase of $133.9 million from 2022. This includes cost
of goods sold of $827.9 million, selling, general, and administrative expenses of $264.8
million, and research and development expenses of $15.0 million. Net Income: The company
reported a net income of $1,027.7 million in 2023, a decline of $233.4 million from the
previous year. Possible Risks: Some potential risks that could impact the company's
financial performance in the future include economic downturns, increased competition, and
supply chain disruptions. Additionally, the company's reliance on a few major customers
and suppliers may pose a risk i

In [18]:
print('GPU Usage after the retrieval 2st prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 2st prompt:
| ID | GPU | MEM |
------------------
|  0 | 91% | 51% |


In [19]:
result = qa_chain(
    "What were the net incomes from continuing operations of the company in the period from 2021 to 2023?"
)
print(fill(result["result"].strip(), width=90))

Based on the information provided in the annual report, the net income from continuing
operations of the company in each of the periods from 2021 to 2023 is as follows: * For
fiscal year 2021, the net income from continuing operations was $2,281.4 million. * For
fiscal year 2022, the net income from continuing operations was $2,338.8 million. * For
fiscal year 2023, the net income from continuing operations was $2,494.6 million.
Therefore, the net income from continuing operations increased by $157.4 million from 2021
to 2022 and further increased by $163.2 million from 2022 to 2023.


In [20]:
print('GPU Usage after the retrieval 3st prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 3st prompt:
| ID | GPU | MEM |
------------------
|  0 | 89% | 53% |


In [21]:
result = qa_chain(
    "What were earnings per share (EPS) in 2022-2023 period?"
)
print(fill(result["result"].strip(), width=90))

Based on the provided data, the earnings per share (EPS) for the 2022-2023 period was
$10.33.


In [22]:
print('GPU Usage after the retrieval 4th prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 4th prompt:
| ID | GPU | MEM |
------------------
|  0 | 92% | 55% |


In [23]:
result = qa_chain(
    "What was the difference between total current assets and total current liabilities in 2023?"
)
print(fill(result["result"].strip(), width=90))

Based on the information provided in the annual report, the difference between total
current assets and total current liabilities in 2023 is: Total Current Assets in 2023 =
$36.2 + $83.8 + $36.4 + $97.5 = $254.8 million Total Current Liabilities in 2023 = $5.3 +
$0.6 + $5.8 + $0.4 = $6.7 million Therefore, the difference between total current assets
and total current liabilities in 2023 is $248.1 million ($254.8 - $6.7).
